In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mode
from sklearn import tree
from sklearn.model_selection import KFold, train_test_split

In [11]:
iris = pd.read_csv("iris.data.csv")
data = iris.iloc[ : , 0:4 ]
targ = iris.iloc[ : , 4: ] 
train_d, test_d, train_t, test_t = train_test_split(data, targ, test_size=0.3)
dic = {'Iris-setosa':0 , 'Iris-versicolor':1 , 'Iris-virginica':2}
# print(test_d)

In [12]:
# map targets into integer index
train_tid, test_tid = [], []
for i in range(len(train_t)):
    train_tid.append( dic[ train_t.values[i][0] ] )
for i in range(len(test_t)):
    test_tid.append( dic[ test_t.values[i][0] ] )

In [13]:
# select two features per tree
ord = [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]

In [14]:
# Resubstitution
rf_rs = []
scr_rs = [0] * len(ord)

# training
for i in range(len(ord)):
    dtr = tree.DecisionTreeClassifier()
    dtr = dtr.fit(train_d.iloc[ : , ord[i] ], train_tid)
    # validation
    scr_rs[i] = dtr.score(train_d.iloc[ : , ord[i] ], train_tid)
    rf_rs.append(dtr)
    
# testing
pr_rs = []
for i in range(len(ord)):
    pred = rf_rs[i].predict( test_d.iloc[ : , ord[i] ] )
    for j in range(len(pred)):
        try: pr_rs[j].append( pred[j] )
        except: pr_rs.append( [ pred[j] ] )
            
final_pred = []
for i in range(len(pr_rs)):
    try: final_pred.append(mode(pr_rs[i]))
    except: final_pred.append('')
# print(final_pred)

# confusion matrix
confmx = pd.DataFrame( data=np.zeros((3, 3), dtype=int), columns=['Setosa (p)', 'Versicolor (p)', 'Virginica (p)'], index=['Setosa (t)', 'Versicolor (t)', 'Virginica (t)'] )
for i in range(len(final_pred)):
    try: confmx.iloc[ test_tid[i], final_pred[i] ] += 1
    except: continue

# precision/recall
prmx = pd.DataFrame( data=np.zeros((3, 2), dtype=int), columns=['precision', 'recall'], index=['Setosa', 'Versicolor', 'Virginica'] )
for i in range(len(prmx.index)):
    prmx.iloc[i, 0] = confmx.iloc[i, i] / sum(confmx.iloc[ : , i ]) # precision
    prmx.iloc[i, 1] = confmx.iloc[i, i] / sum(confmx.iloc[ i, : ]) # recall

confmx.style

,Setosa (p),Versicolor (p),Virginica (p)
Setosa (t),12,0,0
Versicolor (t),0,12,1
Virginica (t),0,3,17


In [15]:
prmx.style

,precision,recall
Setosa,1,1
Versicolor,0.8,0.923077
Virginica,0.944444,0.85


In [16]:
# dead function
def draw(X, Y, clf, ):    
    # pairidx: index of order(start from 0), pair: index of chosen columns
    # We only take the two corresponding features
    X = iris.data[:, pair]
    Y = iris.target

    # Train
    clf = DecisionTreeClassifier().fit(X, Y)

    # Plot the decision boundary
    plt.subplot(2, 3, pairidx + 1)

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    #print(Z)
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)

    plt.xlabel(iris.feature_names[pair[0]])
    plt.ylabel(iris.feature_names[pair[1]])



    # Plot the training points
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(Y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i],
                    cmap=plt.cm.RdYlBu, edgecolor='black', s=15)

######## out of loop #######

# n_classes = 3
# plot_colors = "ryb"
# plot_step = 0.02


# plt.suptitle("Decision surface of a decision tree using paired features")
# plt.legend(loc='lower right', borderpad=0, handletextpad=0)
# plt.axis("tight")

# plt.show()

In [8]:
# K-fold
kf = KFold(n_splits=10)

# training
rf_kf = []
scr_kf = [0] * len(ord)
for i in range(len(ord)):
    for tr, val in kf.split(train_d):
        dtr = tree.DecisionTreeClassifier()
        dtr = dtr.fit( train_d.iloc[tr, ord[i]], train_t.iloc[tr] )
        # validation
        scr = dtr.score( train_d.iloc[val, ord[i]], train_t.iloc[val] )
        if scr > scr_kf[i]:
            scr_kf[i] = scr
            try: rf_kf[i] = dtr
            except: rf_kf.append(dtr)
                
# testing
pr_kf = []
for i in range(len(ord)):
    pred = rf_kf[i].predict( test_d.iloc[ : , ord[i] ] )
    for j in range(len(pred)):
        try: pr_kf[j].append( pred[j] )
        except: pr_kf.append( [ pred[j] ] )
            
final_pred = []
for i in range(len(pr_kf)):
    try: final_pred.append(mode(pr_kf[i]))
    except: final_pred.append('')
# print(final_pred)

confmx = pd.DataFrame( data=np.zeros((3, 3), dtype=int), columns=['Setosa (p)', 'Versicolor (p)', 'Virginica (p)'], index=['Setosa (t)', 'Versicolor (t)', 'Virginica (t)'] )
for i in range(len(final_pred)):
    try: confmx.iloc[ test_tid[i], dic[ final_pred[i] ] ] += 1
    except: continue

# precision/recall
prmx = pd.DataFrame( data=np.zeros((3, 2), dtype=int), columns=['precision', 'recall'], index=['Setosa', 'Versicolor', 'Virginica'] )
for i in range(len(prmx.index)):
    prmx.iloc[i, 0] = confmx.iloc[i, i] / sum(confmx.iloc[ : , i ]) # precision
    prmx.iloc[i, 1] = confmx.iloc[i, i] / sum(confmx.iloc[ i, : ]) # recall

confmx.style

,Setosa (p),Versicolor (p),Virginica (p)
Setosa (t),18,0,0
Versicolor (t),0,11,2
Virginica (t),0,2,12


In [9]:
prmx.style

,precision,recall
Setosa,1,1
Versicolor,0.846154,0.846154
Virginica,0.857143,0.857143
